In [1]:
import pandas as pd
import numpy as np

In [2]:
# importing the data set
df1 = pd.read_json('./Daily_Demand_Forecast_Generation_Interchange_2022-01-01_2022-01-31.json')

In [3]:
df1.head()

,period,respondent,respondent-name,type,type-name,timezone,timezone-description,value,value-units
0,2022-01-31,NW,Northwest,DF,Day-ahead demand forecast,Mountain,Mountain,989312,megawatthours
1,2022-01-31,ISNE,ISO New England,TI,Total interchange,Mountain,Mountain,-66894,megawatthours
2,2022-01-31,ISNE,ISO New England,TI,Total interchange,Pacific,Pacific,-66835,megawatthours
3,2022-01-31,ISNE,ISO New England,TI,Total interchange,Arizona,Arizona,-66894,megawatthours
4,2022-01-31,ISNE,ISO New England,TI,Total interchange,Central,Central,-66738,megawatthours


In [4]:
# considering only the New York Independent System Operator
df1=df1[df1['respondent-name']=='New York Independent System Operator']


In [5]:
df1.head()

,period,respondent,respondent-name,type,type-name,timezone,timezone-description,value,value-units
78,2022-01-31,NYIS,New York Independent System Operator,DF,Day-ahead demand forecast,Central,Central,465377,megawatthours
257,2022-01-31,NYIS,New York Independent System Operator,NG,Net generation,Eastern,Eastern,376722,megawatthours
428,2022-01-31,NYIS,New York Independent System Operator,DF,Day-ahead demand forecast,Mountain,Mountain,465358,megawatthours
473,2022-01-31,NYIS,New York Independent System Operator,TI,Total interchange,Eastern,Eastern,-96030,megawatthours
545,2022-01-31,NYIS,New York Independent System Operator,DF,Day-ahead demand forecast,Pacific,Pacific,465320,megawatthours


In [6]:
# listing the unique type-names from the column type-name which will be used as our variables
var = list(df1['type-name'].unique())

In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 634 entries, 78 to 44515
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   period                634 non-null    object
 1   respondent            634 non-null    object
 2   respondent-name       634 non-null    object
 3   type                  634 non-null    object
 4   type-name             634 non-null    object
 5   timezone              634 non-null    object
 6   timezone-description  634 non-null    object
 7   value                 634 non-null    int64 
 8   value-units           634 non-null    object
dtypes: int64(1), object(8)
memory usage: 49.5+ KB


In [8]:
# Use one hot encoding to convert the type-names to variables, for each variable it will generate 1 and O s 
encoded_data = pd.get_dummies(df1,prefix=None,columns=['type-name'])

In [9]:
encoded_data.head()
#encoded_data.info()

,period,respondent,respondent-name,type,timezone,timezone-description,value,value-units,type-name_Day-ahead demand forecast,type-name_Demand,type-name_Net generation,type-name_Total interchange
78,2022-01-31,NYIS,New York Independent System Operator,DF,Central,Central,465377,megawatthours,1,0,0,0
257,2022-01-31,NYIS,New York Independent System Operator,NG,Eastern,Eastern,376722,megawatthours,0,0,1,0
428,2022-01-31,NYIS,New York Independent System Operator,DF,Mountain,Mountain,465358,megawatthours,1,0,0,0
473,2022-01-31,NYIS,New York Independent System Operator,TI,Eastern,Eastern,-96030,megawatthours,0,0,0,1
545,2022-01-31,NYIS,New York Independent System Operator,DF,Pacific,Pacific,465320,megawatthours,1,0,0,0


In [10]:
#here we are converting the 1s to the value generated from the value columns
for item in var:
    print(item)
    encoded_data['type-name_'+item]=encoded_data[['type-name_'+item,'value']].apply(lambda x:x['value'] if x['type-name_'+item] == 1 else x['type-name_'+item], axis=1)

Day-ahead demand forecast
Net generation
Total interchange
Demand


In [11]:
encoded_data.head()

,period,respondent,respondent-name,type,timezone,timezone-description,value,value-units,type-name_Day-ahead demand forecast,type-name_Demand,type-name_Net generation,type-name_Total interchange
78,2022-01-31,NYIS,New York Independent System Operator,DF,Central,Central,465377,megawatthours,465377,0,0,0
257,2022-01-31,NYIS,New York Independent System Operator,NG,Eastern,Eastern,376722,megawatthours,0,0,376722,0
428,2022-01-31,NYIS,New York Independent System Operator,DF,Mountain,Mountain,465358,megawatthours,465358,0,0,0
473,2022-01-31,NYIS,New York Independent System Operator,TI,Eastern,Eastern,-96030,megawatthours,0,0,0,-96030
545,2022-01-31,NYIS,New York Independent System Operator,DF,Pacific,Pacific,465320,megawatthours,465320,0,0,0


In [12]:
# converting the period to date using datetime from pandas
encoded_data['date'] = pd.to_datetime(encoded_data['period'])

In [14]:
# var_new are the new columns generated by the one hot encoding
var_new = []
for item in var:
    var_new.append('type-name_'+item)
    
# grouping the data for each day using the groupby function
data_final = encoded_data.groupby(encoded_data.date.dt.date)[var_new].sum()

In [15]:
# renaming the new columns to the original variables
data_final.columns = var

In [16]:
data_final.head()

,Day-ahead demand forecast,Net generation,Total interchange,Demand
date,,,,
2022-01-01,1739283,1455168,-366759,1821927
2022-01-02,1827741,1467853,-455630,1923483
2022-01-03,2139653,1727063,-533803,2260866
2022-01-04,859357,2125131,-584043,3613710
2022-01-05,2127166,1657137,-526654,2183791


In [86]:
#Save the data
data_final.to_csv('New_York_Independent_System_Operator_Jan_2022.csv', index=True)